In [ ]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

ckpt_converter = '../tts/openvoice_models_downloaded/checkpoints_v2/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs_v2'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')


os.makedirs(output_dir, exist_ok=True)

/opt/homebrew/Caskroom/miniconda/base/envs/muc_project311/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 34368150-8cd3-4df4-b16e-60055a20a427)')' thrown while requesting HEAD https://huggingface.co/M4869/WavMark/resolve/main/step59000_snr39.99_pesq4.35_BERP_none0.30_mean1.81_std1.81.model.pkl


Loaded checkpoint 'openvoice_models_downloaded/checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []


In [ ]:
reference_speaker = './example_reference_elon.mp3' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=True)

OpenVoice version: v2
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[(0.0, 10.579625)]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(t

In [18]:
audio_name

'example_reference_elon_v2_bryLbsiECdMIFitO'

In [ ]:
from melo.api import TTS

long_text = """
So, umm, I was thinking about what makes a voice truly human. It's not just the words, right? Like, if I say, 'Hmm, that's an interesting point,' that 'hmm' carries a lot of meaning. It's a thoughtful pause, a moment of consideration.

And then there are the fillers. You know, like, 'We need to, uhh, re-evaluate our strategy.' That 'uhh' isn't just wasted breath; it often signals that the speaker is searching for the right words, or maybe slightly uncertain.

Sometimes it's a quick intake of breath, a little 'agh' of surprise or mild annoyance. Or a sigh – a deep 'aaah,' conveying relief or perhaps exasperation. It's subtle, but we interpret it instantly.

Even a simple 'oh!' can be, well, varied. Is it 'Oh, really?' with rising surprise, or 'Oh, no...' with a falling, disappointed tone? These tiny vocalizations, the pitch shifts, the slight hesitations, the emphasis on certain syllables... they're the glue that gives our conversations texture and, uhm, personality. A lot of personality, actually.

And let's not forget the expressive sounds – a quick 'shhh' for silence, or a guttural 'ugh' of frustration. A genuine laugh, or a little, almost unconscious, 'hehe' when something's mildly amusing. These aren't words, but they communicate volumes. It's fascinating, isn't it? The sheer complexity of just talking.

"""
texts = {
    'EN_NEWEST': long_text,  # The newest English base speaker model
    # 'EN': "Did you ever hear a folk tale about a giant turtle?",
    # 'ES': "El resplandor del sol acaricia las olas, pintando el cielo con una paleta deslumbrante.",
    # 'FR': "La lueur dorée du soleil caresse les vagues, peignant le ciel d'une palette éblouissante.",
    # 'ZH': "在这次vacation中，我们计划去Paris欣赏埃菲尔铁塔和卢浮宫的美景。",
    # 'JP': "彼は毎朝ジョギングをして体を健康に保っています。",
    # 'KR': "안녕하세요! 오늘은 날씨가 정말 좋네요.",
}


src_path = f'{output_dir}/original_nuances.wav'

# Speed is adjustable
speed = 1.0

for language, text in texts.items():
    model = TTS(language=language, device=device)
    speaker_ids = model.hps.data.spk2id
    
    for speaker_key in speaker_ids.keys():
        speaker_id = speaker_ids[speaker_key]
        speaker_key = speaker_key.lower().replace('_', '-')
        
        source_se = torch.load(f'../tts/openvoice_models_downloaded/checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)
        if torch.backends.mps.is_available() and device == 'cpu':
            torch.backends.mps.is_available = lambda: False
        model.tts_to_file(text, speaker_id, src_path, speed=speed)
        save_path = f'./output_v2_nuances_{speaker_key}.wav'

        # Run the tone color converter
        encode_message = "@MyShell"
        tone_color_converter.convert(
            audio_src_path=src_path, 
            src_se=source_se, 
            tgt_se=target_se, 
            output_path=save_path,
            message=encode_message)



/opt/homebrew/Caskroom/miniconda/base/envs/muc_project311/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


 > Text split to sentences.
So, umm, I was thinking about what makes a voice truly human. It's not just the words, right? Like, if I say, 'Hmm, that's an interesting point, ' that 'hmm' carries a lot of meaning. It's a thoughtful pause, a moment of consideration. And then there are the fillers.
You know, like, 'We need to, uhh, re-evaluate our strategy. ' That 'uhh' isn't just wasted breath; it often signals that the speaker is searching for the right words, or maybe slightly uncertain. Sometimes it's a quick intake of breath, a little 'agh' of surprise or mild annoyance.
Or a sigh – a deep 'aaah, ' conveying relief or perhaps exasperation. It's subtle, but we interpret it instantly. Even a simple 'oh! ' can be, well, varied. Is it 'Oh, really? ' with rising surprise, or 'Oh, no. . . ' with a falling, disappointed tone? These tiny vocalizations,
the pitch shifts, the slight hesitations, the emphasis on certain syllables. . . they're the glue that gives our conversations texture and, uh

100%|██████████| 5/5 [00:56<00:00, 11.36s/it]


In [17]:
model

TTS(
  (model): SynthesizerTrn(
    (enc_p): TextEncoder(
      (emb): Embedding(112, 192)
      (tone_emb): Embedding(11, 192)
      (language_emb): Embedding(4, 192)
      (bert_proj): Conv1d(1024, 192, kernel_size=(1,), stride=(1,))
      (ja_bert_proj): Conv1d(768, 192, kernel_size=(1,), stride=(1,))
      (encoder): Encoder(
        (spk_emb_linear): Linear(in_features=256, out_features=192, bias=True)
        (drop): Dropout(p=0.1, inplace=False)
        (attn_layers): ModuleList(
          (0-5): 6 x MultiHeadAttention(
            (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
            (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
            (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
            (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
            (drop): Dropout(p=0.1, inplace=False)
          )
        )
        (norm_layers_1): ModuleList(
          (0-5): 6 x LayerNorm()
        )
        (ffn_layers): ModuleList(
 

In [20]:
# TTS(language=language, device=device)
language

'ZH'